# API Forever21

In [ ]:
import requests
import pandas as pd
import json
import os
import sys
import dotenv

dotenv.load_dotenv()


sys.path.append(os.path.abspath('../'))
from src import funciones_api as fa
from src import dic_colores

Obtenemos todos los vestidos de la tienda:

In [ ]:
url = "https://apidojo-forever21-v1.p.rapidapi.com/products/search"
api_key = os.getenv("api_forever21")

querystring = {"query":"dress","rows":"1000","start":"0"}

headers = {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "apidojo-forever21-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

res = response.json()
with open("../datos/api_forever21/vestidos_21.json", 'w') as file:
    json.dump(res, file, indent=4)

Para cada vestido sacamos un json más detallado:

In [2]:
lista_id_vestido_21 = []
with open("../datos/api_forever21/vestidos_21.json", 'r') as file:
    vestidos_forever21 = json.load(file)

for i in range(len(vestidos_forever21["response"]["docs"])):
    id = vestidos_forever21["response"]["docs"][i]["pid"]
    lista_id_vestido_21.append(id)

In [ ]:
try:	
	for id in lista_id_vestido_21:
		url = "https://apidojo-forever21-v1.p.rapidapi.com/products/v2/detail"
		api_key = os.getenv("api_forever21")

		querystring = {"productId":{id}}

		headers = {
			"x-rapidapi-key": api_key,
			"x-rapidapi-host": "apidojo-forever21-v1.p.rapidapi.com"
		}

		response = requests.get(url, headers=headers, params=querystring)

		producto = response.json()
		with open(f"../datos/api_forever21/{id}_forever21.json", 'w') as file:
			json.dump(producto, file, indent=4)

except requests.exceptions.RequestException as e:
    print(f"Error de red al solicitar el producto {id}: {e}")
except json.JSONDecodeError as e:
    print(f"Error al decodificar JSON para el producto {id}: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado con el producto {id}: {e}")

Una vez tenemos el json de todos los vestidos extraemos la informacion relevante:

In [21]:
dic_vestidos = fa.creacion_diccionario_forever21()
df_forever21 = pd.DataFrame(dic_vestidos)

In [22]:
# cambiamos los colores a español
df_forever21["color"] = df_forever21["color"].map(dic_colores.colores)

# añadimos la columna de categoria del vestido
df_forever21["nombre"] = df_forever21["nombre"].str.lower()
df_forever21["categoria"] = df_forever21["nombre"].str.extract('(largo|midi|corto|maxi|mini)', expand=True)

# Estandarizamos las tallas
dic_tallas = {"0X":"XXL", "1X":"3XL", "2X":"4XL", "3X":"5XL", "4X":"6XL", 'XS':'XS', 'S':'S', 'M':'M', 'L':'L', 'XL':'XL', 'XXL':'XXL'}
df_forever21["talla"] = df_forever21["talla"].map(dic_tallas)

Una vez comprobado que no hay duplicados y nulos solo en la categoria creada por nosotros, procedemos a almacenar el dataframe.

In [23]:
df_forever21.to_csv("../datos/dataframes/df_forever21.csv")

In [26]:
df_forever21.sample(2)

,nombre,marca,precio,color,talla,stock,categoria
314,sleeveless blazer mini dress,Forever21,44.99,negro,M,96.0,mini
188,faux gem cutout midi sweater dress,Forever21,49.99,negro,S,90.0,midi
